In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"poojjja","key":"5bd85524150588a21f8c5a7de2368604"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [ ]:
!kaggle datasets download -d emmarex/plantdisease --unzip


Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 99% 650M/658M [00:03<00:00, 200MB/s]
100% 658M/658M [00:04<00:00, 170MB/s]


In [ ]:
import os


dataset_dir = 'plantdisease'
print("Dataset structure:")
for root, dirs, files in os.walk(dataset_dir):
    level = root.replace(dataset_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 2 * (level + 1)
    for d in sorted(dirs)[:5]:
        print(f"{sub_indent}{d}")
    if len(dirs) > 5:
        print(f"{sub_indent}... and {len(dirs)-5} more classes")
    break


Dataset structure:


In [ ]:
import os, shutil
from sklearn.model_selection import train_test_split

BASE_DIR   = 'plantvillage/PlantVillage'
TRAIN_DIR  = 'plantvillage/train'
VAL_DIR    = 'plantvillage/val'

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)


for cls in os.listdir(BASE_DIR):
    cls_path = os.path.join(BASE_DIR, cls)
    if not os.path.isdir(cls_path):
        continue

    images = [f for f in os.listdir(cls_path)
              if f.lower().endswith(('.jpg','.jpeg','.png'))]
    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(TRAIN_DIR, cls), exist_ok=True)
    os.makedirs(os.path.join(VAL_DIR, cls), exist_ok=True)

    for f in train_imgs:
        shutil.copy(os.path.join(cls_path, f),
                    os.path.join(TRAIN_DIR, cls, f))
    for f in val_imgs:
        shutil.copy(os.path.join(cls_path, f),
                    os.path.join(VAL_DIR, cls, f))

print("Done splitting into train and val.")


Done splitting into train and val.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

class_names = list(train_gen.class_indices.keys())
print(class_names)


Found 16504 images belonging to 15 classes.
Found 4134 images belonging to 15 classes.
['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 228s 418ms/step - accuracy: 0.2748 - loss: 5.8580 - val_accuracy: 0.1967 - val_loss: 9.0425
Epoch 2/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 207s 401ms/step - accuracy: 0.4233 - loss: 1.9273 - val_accuracy: 0.1955 - val_loss: 5.8292
Epoch 3/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 206s 400ms/step - accuracy: 0.4793 - loss: 1.6864 - val_accuracy: 0.3358 - val_loss: 10.3148
Epoch 4/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 206s 399ms/step - accuracy: 0.5214 - loss: 1.5018 - val_accuracy: 0.4910 - val_loss: 1.5398
Epoch 5/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 217s 420ms/step - accuracy: 0.5513 - loss: 1.4076 - val_accuracy: 0.4122 - val_loss: 7.3812
Epoch 6/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 215s 417ms/step - accuracy: 0.5777 - loss: 1.2944 - val_accuracy: 0.4722 - val_loss: 2.0098
Epoch 7/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 215s 417ms/step - accuracy: 0.5955 - loss: 1.2434 - val_accuracy: 0.6420 - val_loss: 1.2086
Epoch 8/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 216s 418ms/step - accuracy: 0.6355 

In [ ]:
import numpy as np, cv2
from tensorflow.keras.models import load_model

model = load_model('plant_disease_model.h5')

disease_recommendations = {
    'Pepper__bell___Bacterial_spot': 'Remove infected leaves, use copper-based fungicide.',
    'Pepper__bell___healthy': 'Leaf is healthy. No action needed.',
    'Potato___Early_blight': 'Apply protectant fungicide; remove lower infected leaves.',
    'Potato___Late_blight': 'Destroy infected plants, apply systemic fungicide.',
    'Potato___healthy': 'Healthy potato leaf. Maintain normal care.',
    'Tomato_Bacterial_spot': 'Use copper spray; avoid overhead irrigation.',
    'Tomato_Early_blight': 'Apply chlorothalonil or mancozeb; improve airflow.',
    'Tomato_Late_blight': 'Urgent: remove plants, apply mancozeb/metalaxyl.',
    'Tomato_Leaf_Mold': 'Reduce humidity; use fungicide labeled for leaf mold.',
    'Tomato_Septoria_leaf_spot': 'Remove affected leaves; apply foliar fungicide.',
}

def predict_disease(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    img = img.astype('float32')/255.0
    img = np.expand_dims(img, axis=0)

    probs = model.predict(img)[0]
    idx = np.argmax(probs)
    pred_class = class_names[idx]
    conf = probs[idx]


    is_healthy = 'healthy' in pred_class.lower()
    distance = 0.0 if is_healthy else (1 - conf)

    print(f"Class: {pred_class}")
    print(f"Confidence: {conf:.2f}")
    print(f"Disease distance: {distance:.2f}")
    print("Recommendation:",
          disease_recommendations.get(pred_class, 'Consult an expert for management.'))

    return pred_class, distance


In [ ]:
predict_disease('/content/Screenshot 2025-12-27 102326.png')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Class: Pepper__bell___Bacterial_spot
Confidence: 0.95
Disease distance: 0.05
Recommendation: Remove infected leaves, use copper-based fungicide.


('Pepper__bell___Bacterial_spot', np.float32(0.04639709))